In [1]:
import torch
import os
import subprocess

repo_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).decode().strip()
os.chdir(repo_root)

print("cwd =", os.getcwd())

from solvers.torch_eit_fem_solver import EIT
from solvers.torch_eit_fem_solver.fem import Mesh, V_h, dtn_map
from solvers.torch_eit_fem_solver.utils import dtn_from_sigma
from torchdiffeq import odeint
import matplotlib.pyplot as plt
import cmocean as cmo
from scipy import io as sio
from tqdm import tqdm
from models.unet import Unet
from collections import OrderedDict
import numpy as np

device = 'cuda:1' 

dataset = torch.load('data/eit-circles-dtn-default-128.pt', map_location='cpu')

data_root = 'mesh-data'
mesh_file = 'mesh_128_h05.mat'
mat_fname = os.path.join(data_root, mesh_file)
mat_contents = sio.loadmat(mat_fname)

p = torch.tensor(mat_contents['p'], dtype=torch.float64).to(device)
t = torch.tensor(mat_contents['t']-1, dtype=torch.long).to(device)
vol_idx = torch.tensor(mat_contents['vol_idx'].reshape((-1,))-1, dtype=torch.long).to(device)
bdy_idx = torch.tensor(mat_contents['bdy_idx'].reshape((-1,))-1, dtype=torch.long).to(device)

mesh = Mesh(p, t, bdy_idx, vol_idx)
v_h = V_h(mesh)

dtn_background = dtn_from_sigma(sigma_vec=torch.ones(128, 128), v_h=v_h, mesh=mesh, img_size=128, device=device).cpu()

train_X = dataset['train']['dtn_map'].float()
train_X /= dtn_background
train_Y = dataset['train']['media'].float()

train_X_min = train_X.min()
train_X_max = train_X.max()
train_Y_min = train_Y.min()
train_Y_max = train_Y.max()

cwd = /home/johnma/flow-inverse


In [2]:
from models.unet import Unet
model = Unet(ch=32).to(device)
checkpoint = torch.load("saved_runs/circles-eit-inverse-unet-interp-y-pred-default-bg-transform/checkpoints/ckp_32750.tar", map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Unet(
  (input_proj): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): SiLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
  (down): ModuleList(
    (0): ModuleList(
      (0-1): 2 x ResBlock(
        (skip_connection): Identity()
        (block1): Sequential(
          (0): GroupNorm(32, 32, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Identity()
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (block2): Sequential(
          (0): GroupNorm(32, 32, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (cond_block): ConditioningBlock(
          (proj): Sequential(
            (0): SiLU()
            (1): Linear(in_features=128, out_features=64, bias=True)


In [ ]:
test_X = dataset['test']['dtn_map'].float()


def interpolate_xt(t: torch.Tensor, x0: torch.Tensor, x1: torch.Tensor) -> torch.Tensor:
    # broadcast t
    t = t[:, None, None, None]
    # interpolate
    return t*x1 + (1 - t)*x0


dtn = test_X[0]
dtn = dtn / background
dtn = 2.0 * (dtn - train_X_min) / (train_X_max - train_X_min + 1e-12) - 1.0

t = [0, 0.25, 0.50, 0.75, 1.0]

t = torch.tensor(t, dtype=torch.float32)

results = {}

for i in range(len(t)):
    Z = torch.randn_like(dtn)
    xt = interpolate_xt(t[i], dtn, Z)
    out = model(xt, t[i])
    results[i] 




NameError: name 'background' is not defined